In [1]:
#In the prototype, we focus on "text recognition" only 
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold=KFold(n_splits=10)

In [14]:
# reading in the CSV files as Pandas dataframes
df_train_ori = pd.read_csv('vizwiz_skill_typ_train.csv', skipinitialspace = True, engine='python')
df_val_ori = pd.read_csv('vizwiz_skill_typ_val.csv', skipinitialspace = True, engine='python')
# The structure of VizWiz file
df_train_ori.head(3)
df_val_ori.head(3)

,qid,IMG,QSN,ANS_TYP,TXT,OBJ,COL,CNT,OTH,ANS1,ANS2,ANS3,ANS4,ANS5,ANS6,ANS7,ANS8,ANS9,ANS10
0,VizWiz_val_000000028002.jpg,VizWiz_val_000000028002.jpg,What is this?,other,0,1,0,0,0,unsuitable,unanswerable,unanswerable,paper,person,women holding arm up,persons chest,person,t shirt,t shirt
1,VizWiz_val_000000028003.jpg,VizWiz_val_000000028003.jpg,What is this?,unanswerable,1,1,0,0,0,unsuitable,lotion,unanswerable,unsuitable,body lotion,lotion,gold tube,unsuitable,make up,lotion
2,VizWiz_val_000000028005.jpg,VizWiz_val_000000028005.jpg,What is this?,other,0,1,0,0,0,coke bottle cap,shirt pin,coca cola coin,bottle cap,coca cola pin,bottle cap,coca cola 1940s metal disk,cocoa cola 1940 coin,coca cola metal cap,not clare


In [9]:
# reading in the CSV files as Pandas dataframes
api_feature_train = pd.read_csv('api_features_train.csv', skipinitialspace = True, engine='python')
api_feature_val = pd.read_csv('api_features_val.csv', skipinitialspace = True, engine='python')
nn_feature_train = pd.read_csv('nn_features_train.csv', skipinitialspace = True, engine='python')
nn_feature_val = pd.read_csv('nn_features_val.csv', skipinitialspace = True, engine='python')
# The structure of VizWiz file
api_feature_train.head(3)

,qid,adult,categories,descriptions,tags
0,VizWiz_train_000000000000.jpg,False,others_,a bottle of wine on a table,'bottle' 'indoor' 'table' 'sitting' 'wine'...
1,VizWiz_train_000000000001.jpg,False,others_,a kitchen with a sink and a microwave,'indoor' 'cabinet' 'kitchen' 'small' 'refr...
2,VizWiz_train_000000000002.jpg,False,others_,a can of soda,'bottle' 'sitting' 'table' 'orange' 'layin...


In [10]:
# The structure of VizWiz file
nn_feature_train.head(3)

,qid,vgg_tags alexnet_tags,txt,col
0,VizWiz_train_000000000000.jpg,perfume essence packet,1,0
1,VizWiz_train_000000000001.jpg,espresso maker lighter light igniter ignitor,1,0
2,VizWiz_train_000000000002.jpg,pill bottle water bottle,1,0


In [12]:
df_train = pd.merge(df_train_ori, api_feature_train, how='left', on='qid')
df_train = pd.merge(df_train, nn_feature_train, how='left', on='qid')
df_train.head(3)

,qid,IMG,QSN,ANS_TYP,TXT,OBJ,COL,CNT,OTH,ANS1,...,ANS8,ANS9,ANS10,adult,categories,descriptions,tags,vgg_tags alexnet_tags,txt,col
0,VizWiz_train_000000000000.jpg,VizWiz_train_000000000000.jpg,What's the name of this product?,other,1,1,0,0,0,basil leaves,...,basil leaves,basil leaves,basil,False,others_,a bottle of wine on a table,'bottle' 'indoor' 'table' 'sitting' 'wine'...,perfume essence packet,1,0
1,VizWiz_train_000000000000.jpg,VizWiz_train_000000000000.jpg,What's the name of this product?,other,1,1,0,0,0,basil leaves,...,basil leaves,basil leaves,basil,False,others_,a bottle of wine on a table,'bottle' 'indoor' 'table' 'sitting' 'wine'...,lotion packet,1,0
2,VizWiz_train_000000000001.jpg,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,other,1,1,0,0,0,soda,...,coke 0,coca cola,coke,False,others_,a kitchen with a sink and a microwave,'indoor' 'cabinet' 'kitchen' 'small' 'refr...,espresso maker lighter light igniter ignitor,1,0


In [15]:
df_val = pd.merge(df_val_ori, api_feature_val, how='left', on='qid')
df_val = pd.merge(df_val, nn_feature_val, how='left', on='qid')
df_val.head(3)

,qid,IMG,QSN,ANS_TYP,TXT,OBJ,COL,CNT,OTH,ANS1,...,ANS10,adult,categories,descriptions,tags,txt_x,col_x,vgg_tags alexnet_tags,txt_y,col_y
0,VizWiz_val_000000028002.jpg,VizWiz_val_000000028002.jpg,What is this?,other,0,1,0,0,0,unsuitable,...,t shirt,False,others_,None,'person' 'indoor' 'man' 'holding' 'red' '...,1,0,seat belt seatbelt seat belt seatbelt,0,0
1,VizWiz_val_000000028003.jpg,VizWiz_val_000000028003.jpg,What is this?,unanswerable,1,1,0,0,0,unsuitable,...,lotion,False,abstract_,a close up of a yellow wall,'indoor' 'sitting' 'small' 'yellow' 'white',1,0,fountain pen fountain pen,1,0
2,VizWiz_val_000000028005.jpg,VizWiz_val_000000028005.jpg,What is this?,other,0,1,0,0,0,coke bottle cap,...,not clare,False,others_,None,'sitting' 'phone' 'food' 'white',1,0,cassette iPod,0,0


In [35]:
#merge text
df_train["allText1"] = df_train["QSN"].map(str) + df_train["tags"].map(str)
df_train["allText2"] = df_train["QSN"].map(str) + df_train["vgg_tags alexnet_tags"].map(str)
df_train["allText3"] = df_train["QSN"].map(str) + df_train["descriptions"].map(str)
df_train.head(3)
df_val["allText1"] = df_val["QSN"].map(str) + df_val["tags"].map(str)
df_val["allText2"] = df_val["QSN"].map(str) + df_val["vgg_tags alexnet_tags"].map(str)
df_val["allText3"] = df_val["QSN"].map(str) + df_val["descriptions"].map(str)
df_val.head(3)

,qid,IMG,QSN,ANS_TYP,TXT,OBJ,COL,CNT,OTH,ANS1,...,descriptions,tags,txt_x,col_x,vgg_tags alexnet_tags,txt_y,col_y,allText1,allText2,allText3
0,VizWiz_val_000000028002.jpg,VizWiz_val_000000028002.jpg,What is this?,other,0,1,0,0,0,unsuitable,...,None,'person' 'indoor' 'man' 'holding' 'red' '...,1,0,seat belt seatbelt seat belt seatbelt,0,0,What is this?'person' 'indoor' 'man' 'holdi...,What is this?seat belt seatbelt seat belt se...,What is this?None
1,VizWiz_val_000000028003.jpg,VizWiz_val_000000028003.jpg,What is this?,unanswerable,1,1,0,0,0,unsuitable,...,a close up of a yellow wall,'indoor' 'sitting' 'small' 'yellow' 'white',1,0,fountain pen fountain pen,1,0,What is this?'indoor' 'sitting' 'small' 'ye...,What is this?fountain pen fountain pen,What is this?a close up of a yellow wall
2,VizWiz_val_000000028005.jpg,VizWiz_val_000000028005.jpg,What is this?,other,0,1,0,0,0,coke bottle cap,...,None,'sitting' 'phone' 'food' 'white',1,0,cassette iPod,0,0,What is this?'sitting' 'phone' 'food' 'white',What is this?cassette iPod,What is this?None


In [93]:
#create X and Ys
ques_train=df_train["allText3"].values
txt_train=df_train["TXT"].values
col_train=df_train["COL"].values

ques_val=df_val["allText3"].values
txt_val=df_val["TXT"].values
col_val=df_val["COL"].values


In [94]:
print(ques_train)

["What's the name of this product?a bottle of wine on a table"
 "What's the name of this product?a bottle of wine on a table"
 'Can you tell me what is in this can please?a kitchen with a sink and a microwave'
 ... 'What is the name of this?a screen shot of a computer monitor'
 'What is this?a person in a blue shirt'
 'What is this ?a black computer mouse on a table']


In [95]:
#Use LSTM to predict
import gzip
import numpy as np
import os
import pandas as pd
import pickle
import requests
import time

os.environ['KERAS_BACKEND']='cntk'
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical

In [96]:
MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

In [97]:
# tokenize, create seqs, pad
tok = Tokenizer(num_words=VOCAB_SIZE, lower=True, split=" ")
tok.fit_on_texts(ques_train)
train_seq = tok.texts_to_sequences(ques_train)
train_seq = sequence.pad_sequences(train_seq, maxlen=MAX_DOC_LEN)
val_seq = tok.texts_to_sequences(ques_val)
val_seq = sequence.pad_sequences(val_seq, maxlen=MAX_DOC_LEN)

In [98]:
labels = to_categorical(np.asarray(col_train))
labels = labels.astype('float32')
print('Number of reviews by class in training set')
print(labels.sum(axis=0))
n_classes = labels.shape[1]

Number of reviews by class in training set
[16766. 10205.]


In [99]:
import nltk 

nltk.download('punkt')

sent_lst = []

for doc in ques_train:
    sentences = nltk.tokenize.sent_tokenize(doc)
    for sent in sentences:
        word_lst = [w for w in nltk.tokenize.word_tokenize(sent) if w.isalnum()]
        sent_lst.append(word_lst)

[nltk_data] Downloading package punkt to /home/yanan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [100]:
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# use skip-gram
word2vec_model = gensim.models.Word2Vec(sentences=sent_lst, min_count=6, size=EMBEDDING_DIM, sg=1, workers=os.cpu_count())

2018-12-03 10:18:22,040 : INFO : collecting all words and their counts
2018-12-03 10:18:22,041 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-03 10:18:22,060 : INFO : PROGRESS: at sentence #10000, processed 100370 words, keeping 1877 word types
2018-12-03 10:18:22,078 : INFO : PROGRESS: at sentence #20000, processed 199094 words, keeping 2625 word types
2018-12-03 10:18:22,096 : INFO : PROGRESS: at sentence #30000, processed 298253 words, keeping 3264 word types
2018-12-03 10:18:22,101 : INFO : collected 3467 word types from a corpus of 318321 raw words and 32031 sentences
2018-12-03 10:18:22,101 : INFO : Loading a fresh vocabulary
2018-12-03 10:18:22,106 : INFO : effective_min_count=6 retains 1325 unique words (38% of original 3467, drops 2142)
2018-12-03 10:18:22,106 : INFO : effective_min_count=6 leaves 313308 word corpus (98% of original 318321, drops 5013)
2018-12-03 10:18:22,110 : INFO : deleting the raw counts dictionary of 3467 items
2018-12-

In [101]:
embeddings_index = {}

for word in word2vec_model.wv.vocab:
    coefs = np.asarray(word2vec_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs

print('Total %s word vectors.' % len(embeddings_index))

# Initial embedding
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in tok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and i < VOCAB_SIZE:
        embedding_matrix[i] = embedding_vector

Total 1325 word vectors.


In [102]:
BATCH_SIZE = 50
NUM_EPOCHS = 30
LSTM_DIM = 100
OPTIMIZER = SGD(lr=0.01, nesterov=True)

In [103]:
def lstm_create_train(reg_param):
    l2_reg = regularizers.l2(reg_param)

    # model init
    embedding_layer = Embedding(VOCAB_SIZE,
                                EMBEDDING_DIM,
                                input_length=MAX_DOC_LEN,
                                trainable=True,
                                mask_zero=False,
                                embeddings_regularizer=l2_reg,
                                weights=[embedding_matrix])

    lstm_layer = LSTM(units=LSTM_DIM, kernel_regularizer=l2_reg)
    dense_layer = Dense(n_classes, activation='softmax', kernel_regularizer=l2_reg)

    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(lstm_layer))
    model.add(dense_layer)

    model.compile(loss='categorical_crossentropy',
                  optimizer=OPTIMIZER,
                  metrics=['acc'])

    fname = "lstm"
    history = History()
    csv_logger = CSVLogger('./{0}_{1}.log'.format(fname, reg_param),
                           separator=',',
                           append=True)

    t1 = time.time()
    # model fit
    model.fit(train_seq,
              labels.astype('float32'),
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              callbacks=[history, csv_logger],
              verbose=2)
    t2 = time.time()

    # save model
    model.save('./{0}_{1}_model.h5'.format(fname, reg_param))
    np.savetxt('./{0}_{1}_time.txt'.format(fname, reg_param), 
               [reg_param, (t2-t1) / 3600])
    with open('./{0}_{1}_history.txt'.format(fname, reg_param), "w") as res_file:
        res_file.write(str(history.history))

In [104]:
lstm_create_train(1e-7)

Epoch 1/30
 - 22s - loss: 0.6408 - acc: 0.6310
Epoch 2/30
 - 22s - loss: 0.5994 - acc: 0.6975
Epoch 3/30
 - 22s - loss: 0.5683 - acc: 0.7267
Epoch 4/30
 - 22s - loss: 0.5399 - acc: 0.7491
Epoch 5/30
 - 22s - loss: 0.5265 - acc: 0.7591
Epoch 6/30
 - 22s - loss: 0.5163 - acc: 0.7638
Epoch 7/30
 - 22s - loss: 0.5068 - acc: 0.7689
Epoch 8/30
 - 22s - loss: 0.4985 - acc: 0.7702
Epoch 9/30
 - 22s - loss: 0.4913 - acc: 0.7733
Epoch 10/30
 - 22s - loss: 0.4848 - acc: 0.7742
Epoch 11/30
 - 22s - loss: 0.4824 - acc: 0.7771
Epoch 12/30
 - 22s - loss: 0.4791 - acc: 0.7768
Epoch 13/30
 - 21s - loss: 0.4764 - acc: 0.7777
Epoch 14/30
 - 21s - loss: 0.4748 - acc: 0.7801
Epoch 15/30
 - 21s - loss: 0.4731 - acc: 0.7805
Epoch 16/30
 - 21s - loss: 0.4720 - acc: 0.7809
Epoch 17/30
 - 21s - loss: 0.4706 - acc: 0.7825
Epoch 18/30
 - 21s - loss: 0.4689 - acc: 0.7829
Epoch 19/30
 - 21s - loss: 0.4685 - acc: 0.7829
Epoch 20/30
 - 21s - loss: 0.4668 - acc: 0.7838
Epoch 21/30
 - 22s - loss: 0.4665 - acc: 0.7843
E

In [105]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

model = load_model('./lstm_{0}_model.h5'.format(1e-7))
preds = model.predict(val_seq, verbose=0)
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(col_val, preds.argmax(axis=1)), 
       roc_auc_score(txt_val, preds[:, 1]))))

Accuracy = 0.7842083505580818 	 AUC = 0.1341261639057547
